In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'

# Read Data

In [2]:
def file_reader(basedir, filename):
    if filename.endswith('.csv'):
        try:
            df = pd.read_csv(basedir + filename, header=15)
        except:
            df = pd.read_csv(basedir + filename, encoding='euc-kr', header=15)
    elif filename.endswith('.xlsx'):
        try:
            df = pd.read_excel(basedir + filename, header=16)
        except:
            df = pd.read_excel(basedir + filename, encoding='euc-kr', header=16)
    else:
        print('error')
    
    return df

In [3]:
def read_data():
    basedir = './국토교통부_실거래가_공개시스템/'
    filenames = os.listdir(basedir)
    
    apart_trade_files = [filename for filename in filenames if filename.startswith('아파트(매매)')]
    apart_lease_files = [filename for filename in filenames if filename.startswith('아파트(전월세)')]
    multiplex_trade_files = [filename for filename in filenames if filename.startswith('연립다세대(매매)')]
    multiplex_lease_files = [filename for filename in filenames if filename.startswith('연립다세대(전월세)')]
    officetel_trade_files = [filename for filename in filenames if filename.startswith('오피스텔(매매)')]
    officetel_lease_files = [filename for filename in filenames if filename.startswith('오피스텔(전월세)')]
    
    #
    apart_trade_dfs_list = []
    for filename in apart_trade_files:
        df = file_reader(basedir, filename)
        apart_trade_dfs_list.append(df)
    apart_trade_df = pd.concat(apart_trade_dfs_list).reset_index(drop=True)
    
    #
    apart_lease_dfs_list = []
    for filename in apart_lease_files:
        df = file_reader(basedir, filename)
        apart_lease_dfs_list.append(df)
    apart_lease_df = pd.concat(apart_lease_dfs_list).reset_index(drop=True)
    
    #
    multiplex_trade_dfs_list = []
    for filename in multiplex_trade_files:
        df = file_reader(basedir, filename)
        multiplex_trade_dfs_list.append(df)
    multiplex_trade_df = pd.concat(multiplex_trade_dfs_list).reset_index(drop=True)
    
    #
    multiplex_lease_dfs_list = []
    for filename in multiplex_lease_files:
        df = file_reader(basedir, filename)
        multiplex_lease_dfs_list.append(df)
    multiplex_lease_df = pd.concat(multiplex_lease_dfs_list).reset_index(drop=True)
    
    #
    officetel_trade_dfs_list = []
    for filename in officetel_trade_files:
        df = file_reader(basedir, filename)
        officetel_trade_dfs_list.append(df)
    officetel_trade_df = pd.concat(officetel_trade_dfs_list).reset_index(drop=True)
    
    #
    officetel_lease_dfs_list = []
    for filename in officetel_lease_files:
        df = file_reader(basedir, filename)
        officetel_lease_dfs_list.append(df)
    officetel_lease_df = pd.concat(officetel_lease_dfs_list).reset_index(drop=True)
    
    
    return apart_trade_df, apart_lease_df, multiplex_trade_df, multiplex_lease_df, officetel_trade_df, officetel_lease_df

In [4]:
%%time
apart_trade_df, apart_lease_df, multiplex_trade_df, multiplex_lease_df, officetel_trade_df, officetel_lease_df\
= read_data()

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\IPython\core\magics\execution.py:1321: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


Wall time: 4min 29s


# From Previous EDA & Preprocessing

In [5]:
apart_lease_df.drop(apart_lease_df[apart_lease_df['전용면적(㎡)'].isna()].index, inplace=True)

In [6]:
# 전용면적과 층 null들을 median 값으로 교체 시 사용

def area_floor_fillna_median(df):
    df = df.copy()
    
    df['temp_full_addr'] = df['시군구'].str.split(' ').apply(lambda x: x[0]) + ' ' + df['시군구'].str.split(' ').apply(lambda x: x[1])\
                           + ' ' + df['도로명'] + ' ' + df['단지명']
    
    area_null_df = df[df['전용면적(㎡)'].isna()]
    
    unique_addresses = area_null_df['temp_full_addr'].unique()
    
    for address in unique_addresses:
        addr_df = df[df['temp_full_addr'] == address]
        
        area_median = addr_df['전용면적(㎡)'].median()
        
        df.loc[addr_df[addr_df['전용면적(㎡)'].isna()].index, '전용면적(㎡)'] = area_median
        
    df = df.drop(df[df['전용면적(㎡)'].isna()].index)
        
        
    floor_null_df = df[df['층'].isna()]
    
    unique_addresses = floor_null_df['temp_full_addr'].unique()
    
    for address in unique_addresses:
        addr_df = df[df['temp_full_addr'] == address]
        
        floor_median = addr_df['층'].median()
        
        df.loc[addr_df[addr_df['층'].isna()].index, '층'] = floor_median
        
    df = df.drop(df[df['층'].isna()].index)
        
    return df

In [7]:
multiplex_trade_df.loc[multiplex_trade_df[multiplex_trade_df['건축년도'].isna()].index, '건축년도'] = 1971
multiplex_lease_df.drop(multiplex_lease_df[multiplex_lease_df['층'].isna()].index, inplace=True)

In [8]:
# 모든 df 도로명 빈 칸 null 로 교체

def replace_empty_to_null(df):
    df = df.copy()
    
    empty_df = df[df['도로명'] == ' ']
    
    df.loc[empty_df.index, '도로명'] = np.nan
    
    return df

In [9]:
apart_trade_df = replace_empty_to_null(apart_trade_df)
apart_lease_df = replace_empty_to_null(apart_lease_df)
multiplex_trade_df = replace_empty_to_null(multiplex_trade_df)
multiplex_lease_df = replace_empty_to_null(multiplex_lease_df)
officetel_trade_df = replace_empty_to_null(officetel_trade_df)
officetel_lease_df = replace_empty_to_null(officetel_lease_df)

In [10]:
multiplex_lease_df.loc[multiplex_lease_df[multiplex_lease_df['건축년도'].isna()].index, '건축년도'] = 1971

In [11]:
# 건축년도가 null 인 row 들에 있는 주소들 중에서, 같은 주소지만 건축년도 값이 있는 주소는 null 을 채워넣고, 없는 주소는 df 에서 제거.
# 추가적으로, 한 주소지, 단지이름에 건축년도가 두 가지 이상 있는지도 조사함.

def bltyear_fill_and_remove_null(df):
    df = df.copy()
    
    df['temp_full_addr'] = df['시군구'] + df['번지'] + df['단지명']
    
    bltyear_null_df = df[df['건축년도'].isna()]
    
    unique_addrs = bltyear_null_df['temp_full_addr'].unique()
    
    concat_list = []
    for addr in unique_addrs:
        addr_df = df[df['temp_full_addr'] == addr]
        if addr_df['건축년도'].isna().sum() != addr_df.shape[0]:
            if addr_df['건축년도'].std() != 0:
                print('건축년도가 두 가지 이상인 주소가 있음.')
            
            addr_df['건축년도'].fillna(addr_df['건축년도'].mean(), inplace=True)
            
            concat_list.append(addr_df)
    
    df.drop(columns=['temp_full_addr'], inplace=True)
    
    if len(concat_list) > 0:
        concat_df = pd.concat(concat_list)
    else:
        return df
    
    #return concat_df
    
    df.loc[concat_df.index, '건축년도'] = concat_df['건축년도']
    
    bltyear_null_df = df[df['건축년도'].isna()]
    
    df.drop(bltyear_null_df.index, inplace=True)
        
    return df.reset_index(drop=True)

In [12]:
officetel_trade_df = bltyear_fill_and_remove_null(officetel_trade_df)
officetel_lease_df = bltyear_fill_and_remove_null(officetel_lease_df)

In [13]:
def trade_dfs_change_dtype(df):
    df = df.copy()
    
    df['거래금액(만원)'] = df['거래금액(만원)'].apply(str)
    df['거래금액(만원)'] = pd.to_numeric(df['거래금액(만원)'].str.replace(',', ''))
    df['층'] = df['층'].astype('int')
    df['건축년도'] = df['건축년도'].astype('int')
    
    return df

In [14]:
apart_trade_df = trade_dfs_change_dtype(apart_trade_df)
multiplex_trade_df = trade_dfs_change_dtype(multiplex_trade_df)
officetel_trade_df = trade_dfs_change_dtype(officetel_trade_df)

In [15]:
def lease_dfs_change_dtype(df):
    df = df.copy()
    
    df['보증금(만원)'] = df['보증금(만원)'].apply(str)
    df['월세(만원)'] = df['월세(만원)'].apply(str)
    df['보증금(만원)'] = pd.to_numeric(df['보증금(만원)'].str.replace(',', ''))
    df['월세(만원)'] = pd.to_numeric(df['월세(만원)'].str.replace(',', ''))
    df['층'] = df['층'].astype('int')
    df['건축년도'] = df['건축년도'].astype('int')
    
    return df

In [16]:
apart_lease_df = lease_dfs_change_dtype(apart_lease_df)
multiplex_lease_df = lease_dfs_change_dtype(multiplex_lease_df)
officetel_lease_df = lease_dfs_change_dtype(officetel_lease_df)

In [17]:
def price_per_area_and_year_month(df, mode='trade'):
    df = df.copy()
    
    if mode == 'trade':
        df['가격/면적'] = df['거래금액(만원)'] / df['전용면적(㎡)']
    elif mode == 'lease':
        df['가격/면적'] = df['보증금(만원)'] / df['전용면적(㎡)']
    
    df['year'] = pd.to_numeric(df['계약년월'].astype('str').str[:4])
    df['month'] = pd.to_numeric(df['계약년월'].astype('str').str[4:])
    
    return df    

In [18]:
apart_trade_df = price_per_area_and_year_month(apart_trade_df)
multiplex_trade_df = price_per_area_and_year_month(multiplex_trade_df)
officetel_trade_df = price_per_area_and_year_month(officetel_trade_df)

In [19]:
apart_lease_df = price_per_area_and_year_month(apart_lease_df, mode='lease')
multiplex_lease_df = price_per_area_and_year_month(multiplex_lease_df, mode='lease')
officetel_lease_df = price_per_area_and_year_month(officetel_lease_df, mode='lease')

In [20]:
def si_gu(df):
    df = df.copy()
    
    df['시'] = df['시군구'].str.split(' ').apply(lambda x: x[0])
    df['구'] = df['시군구'].str.split(' ').apply(lambda x: x[1])
    
    return df

In [21]:
apart_trade_df = si_gu(apart_trade_df)
apart_lease_df = si_gu(apart_lease_df)
multiplex_trade_df = si_gu(multiplex_trade_df)
multiplex_lease_df = si_gu(multiplex_lease_df)
officetel_trade_df = si_gu(officetel_trade_df)
officetel_lease_df = si_gu(officetel_lease_df)

In [22]:
apart_lease_deposit_only_df = apart_lease_df[apart_lease_df['전월세구분'] == '전세'].reset_index(drop=True)
apart_lease_monthly_pay_df = apart_lease_df[apart_lease_df['전월세구분'] == '월세'].reset_index(drop=True)
multiplex_lease_deposit_only_df = multiplex_lease_df[multiplex_lease_df['전월세구분'] == '전세'].reset_index(drop=True)
multiplex_lease_monthly_pay_df = multiplex_lease_df[multiplex_lease_df['전월세구분'] == '월세'].reset_index(drop=True)
officetel_lease_deposit_only_df = officetel_lease_df[officetel_lease_df['전월세구분'] == '전세'].reset_index(drop=True)
officetel_lease_monthly_pay_df = officetel_lease_df[officetel_lease_df['전월세구분'] == '월세'].reset_index(drop=True)

# 이상치 (outlier) 제거

* 중요: 이후 이상치에 덜 민감한 알고리즘을 사용할 예정이므로, 이상치 제거 수를 최소한으로 하려고 노력함.

In [23]:
# Tukey의 기법을 사용하여 이상치 (outlier) 검색 후 제거 (가격)

def find_and_remove_Tukey_outliers_price(df, mode='trade'):
    df = df.copy()
    # mode = 'trade' or 'lease'
    
    if mode == 'trade':
        df['target_transformed'] = quantile_transform(df[['거래금액(만원)']], output_distribution='normal', random_state=42)
    elif mode == 'lease':
        df['target_transformed'] = quantile_transform(df[['보증금(만원)']], output_distribution='normal', random_state=42)
    
    q1 = df['target_transformed'].quantile(0.25)
    q3 = df['target_transformed'].quantile(0.75)
    iqr = q3 - q1
    outlier_step = 2 * iqr

    outlier_df = df[(df['target_transformed'] < q1 - outlier_step)]# | (df['target_transformed'] > q3 + outlier_step)]
    
    
    print('{}개의 이상치를 찾아서 제거했습니다.'.format(outlier_df.shape[0]))
    
    df.drop(outlier_df.index, inplace=True)
    df.drop(columns=['target_transformed'], inplace=True)
    df = df.reset_index(drop=True)
    
    return df

In [24]:
apart_trade_df = find_and_remove_Tukey_outliers_price(apart_trade_df)
multiplex_trade_df = find_and_remove_Tukey_outliers_price(multiplex_trade_df)
officetel_trade_df = find_and_remove_Tukey_outliers_price(officetel_trade_df)

48개의 이상치를 찾아서 제거했습니다.
66개의 이상치를 찾아서 제거했습니다.
11개의 이상치를 찾아서 제거했습니다.


In [25]:
apart_lease_deposit_only_df = find_and_remove_Tukey_outliers_price(apart_lease_deposit_only_df, mode='lease')
apart_lease_monthly_pay_df = find_and_remove_Tukey_outliers_price(apart_lease_monthly_pay_df, mode='lease')
multiplex_lease_deposit_only_df = find_and_remove_Tukey_outliers_price(multiplex_lease_deposit_only_df, mode='lease')
multiplex_lease_monthly_pay_df = find_and_remove_Tukey_outliers_price(multiplex_lease_monthly_pay_df, mode='lease')
officetel_lease_deposit_only_df = find_and_remove_Tukey_outliers_price(officetel_lease_deposit_only_df, mode='lease')
officetel_lease_monthly_pay_df = find_and_remove_Tukey_outliers_price(officetel_lease_monthly_pay_df, mode='lease')

114개의 이상치를 찾아서 제거했습니다.
307개의 이상치를 찾아서 제거했습니다.
109개의 이상치를 찾아서 제거했습니다.
409개의 이상치를 찾아서 제거했습니다.
57개의 이상치를 찾아서 제거했습니다.
105개의 이상치를 찾아서 제거했습니다.


In [26]:
# Tukey의 기법을 사용하여 이상치 (outlier) 검색 후 제거 (가격/면적)

def find_and_remove_Tukey_outliers_price_per_area(df):
    df = df.copy()
    
    df['target_transformed'] = quantile_transform(df[['가격/면적']], output_distribution='normal', random_state=42)
    
    q1 = df['target_transformed'].quantile(0.25)
    q3 = df['target_transformed'].quantile(0.75)
    iqr = q3 - q1
    outlier_step = 1.81 * iqr

    outlier_df = df[(df['target_transformed'] < q1 - outlier_step) | (df['target_transformed'] > q3 + outlier_step)]
    
    
    print('{}개의 이상치를 찾아서 제거했습니다.'.format(outlier_df.shape[0]))
    
    df.drop(outlier_df.index, inplace=True)
    df.drop(columns=['target_transformed'], inplace=True)
    df = df.reset_index(drop=True)
    
    return df

In [27]:
apart_trade_df = find_and_remove_Tukey_outliers_price_per_area(apart_trade_df)
multiplex_trade_df = find_and_remove_Tukey_outliers_price_per_area(multiplex_trade_df)
officetel_trade_df = find_and_remove_Tukey_outliers_price_per_area(officetel_trade_df)

1178개의 이상치를 찾아서 제거했습니다.
898개의 이상치를 찾아서 제거했습니다.
217개의 이상치를 찾아서 제거했습니다.


In [28]:
def remove_zero_rows(df):
    df = df.copy()
    
    zero_df = df[df['전용면적(㎡)'] == 0]
    
    print('{}개의 0값을 찾아서 제거했습니다.'.format(zero_df.shape[0]))
    
    df.drop(zero_df.index, inplace=True)
    
    return df

In [29]:
apart_trade_df = remove_zero_rows(apart_trade_df)
multiplex_trade_df = remove_zero_rows(multiplex_trade_df)
officetel_trade_df = remove_zero_rows(officetel_trade_df)
apart_lease_deposit_only_df = remove_zero_rows(apart_lease_deposit_only_df)
apart_lease_monthly_pay_df = remove_zero_rows(apart_lease_monthly_pay_df)
multiplex_lease_deposit_only_df = remove_zero_rows(multiplex_lease_deposit_only_df)
multiplex_lease_monthly_pay_df = remove_zero_rows(multiplex_lease_monthly_pay_df)
officetel_lease_deposit_only_df = remove_zero_rows(officetel_lease_deposit_only_df)
officetel_lease_monthly_pay_df = remove_zero_rows(officetel_lease_monthly_pay_df)

0개의 0값을 찾아서 제거했습니다.
0개의 0값을 찾아서 제거했습니다.
0개의 0값을 찾아서 제거했습니다.
0개의 0값을 찾아서 제거했습니다.
0개의 0값을 찾아서 제거했습니다.
4개의 0값을 찾아서 제거했습니다.
5개의 0값을 찾아서 제거했습니다.
0개의 0값을 찾아서 제거했습니다.
0개의 0값을 찾아서 제거했습니다.


In [30]:
apart_lease_deposit_only_df = find_and_remove_Tukey_outliers_price_per_area(apart_lease_deposit_only_df)
apart_lease_montly_pay_df = find_and_remove_Tukey_outliers_price_per_area(apart_lease_monthly_pay_df)
multiplex_lease_deposit_only_df = find_and_remove_Tukey_outliers_price_per_area(multiplex_lease_deposit_only_df)
multiplex_lease_monthly_pay_df = find_and_remove_Tukey_outliers_price_per_area(multiplex_lease_monthly_pay_df)
officetel_lease_deposit_only_df = find_and_remove_Tukey_outliers_price_per_area(officetel_lease_deposit_only_df)
officetel_lease_monthly_pay_df = find_and_remove_Tukey_outliers_price_per_area(officetel_lease_monthly_pay_df)

1894개의 이상치를 찾아서 제거했습니다.
645개의 이상치를 찾아서 제거했습니다.
438개의 이상치를 찾아서 제거했습니다.
190개의 이상치를 찾아서 제거했습니다.
191개의 이상치를 찾아서 제거했습니다.
176개의 이상치를 찾아서 제거했습니다.


# Concatenating Main Data

In [31]:
apart_trade_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전용면적(㎡)', '계약년월', '계약일', '가격(만원)',
                          '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시', '구']
multiplex_trade_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전용면적(㎡)', '대지권면적(㎡)', '계약년월', '계약일',
                              '가격(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시', '구']
officetel_trade_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전용면적(㎡)', '계약년월', '계약일', '가격(만원)',
                              '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시', '구']

apart_lease_deposit_only_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전월세매매구분', '전용면적(㎡)', '계약년월', '계약일',
                                       '가격(만원)', '월세(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시',
                                       '구']
apart_lease_monthly_pay_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전월세매매구분', '전용면적(㎡)', '계약년월', '계약일',
                                      '가격(만원)', '월세(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시',
                                      '구']
multiplex_lease_deposit_only_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전월세매매구분', '전용면적(㎡)', '계약년월', '계약일',
                                           '가격(만원)', '월세(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시',
                                           '구']
multiplex_lease_monthly_pay_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전월세매매구분', '전용면적(㎡)', '계약년월', '계약일',
                                          '가격(만원)', '월세(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시',
                                          '구']
officetel_lease_deposit_only_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전월세매매구분', '전용면적(㎡)', '계약년월', '계약일',
                                           '가격(만원)', '월세(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시',
                                           '구']
officetel_lease_monthly_pay_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전월세매매구분', '전용면적(㎡)', '계약년월', '계약일',
                                          '가격(만원)', '월세(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시',
                                          '구']

In [32]:
apart_trade_df['전월세매매구분'] = '매매'
apart_trade_df['건물종류'] = '아파트'
multiplex_trade_df['전월세매매구분'] = '매매'
multiplex_trade_df['건물종류'] = '연립다세대'
officetel_trade_df['전월세매매구분'] = '매매'
officetel_trade_df['건물종류'] = '오피스텔'

apart_lease_deposit_only_df['건물종류'] = '아파트'
apart_lease_monthly_pay_df['건물종류'] = '아파트'
multiplex_lease_deposit_only_df['건물종류'] = '연립다세대' 
multiplex_lease_monthly_pay_df['건물종류'] = '연립다세대'
officetel_lease_deposit_only_df['건물종류'] = '오피스텔'
officetel_lease_monthly_pay_df['건물종류'] = '오피스텔'

In [33]:
trade_dfs_list = [apart_trade_df, multiplex_trade_df, officetel_trade_df]
lease_dfs_list = [apart_lease_deposit_only_df, apart_lease_monthly_pay_df,
                  multiplex_lease_deposit_only_df, multiplex_lease_monthly_pay_df,
                  officetel_lease_deposit_only_df, officetel_lease_monthly_pay_df]

In [48]:
def concatenate_main_data(trade_dfs, lease_dfs):
    # trade_dfs, lease_dfs are lists
    trade_dfs = trade_dfs.copy()
    lease_dfs = lease_dfs.copy()    
    
    # 대지권면적 제거
    trade_dfs[1] = trade_dfs[1].drop(columns=['대지권면적(㎡)'])
    
    # 월세 제거
    for i in range(len(lease_dfs)):
        lease_dfs[i] = lease_dfs[i].drop(columns=['월세(만원)'])
    
    #
    dfs_list = []    
    for df in trade_dfs:
        dfs_list.append(df)
        
    for df in lease_dfs:
        dfs_list.append(df)
        
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    return concat_df

In [49]:
main_data_df = concatenate_main_data(trade_dfs_list, lease_dfs_list)
main_data_df.shape

(4668585, 19)

In [50]:
main_data_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,55000,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,매매,아파트
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,58250,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,매매,아파트
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,60000,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,매매,아파트


In [51]:
prac_df = main_data_df.copy()

In [52]:
prac_df['전체주소'] = prac_df['시군구'] + ' ' + prac_df['번지']
prac_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,전체주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,55000,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,58250,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,60000,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1


In [53]:
prac_df['전체주소'].nunique()

109791

# Read Coordinates Data

In [147]:
def read_coor_data1():
    basedir = './좌표데이터/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.txt'):
            try:
                df = pd.read_csv(basedir + filename, sep='|', header=None).drop_duplicates()
            except:
                df = pd.read_csv(basedir + filename, sep='|', header=None, encoding='ansi').drop_duplicates()
                
            return df
            
            df['도로명주소'] = df[3] + ' ' + df[4] + ' ' + df[7] + ' ' + df[9].astype('str') + '-' + df[10].astype('str')
            
            df['x좌표'] = df[16]
            df['y좌표'] = df[17]
            
            df = df[['도로명주소', 'x좌표', 'y좌표']]
            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).drop_duplicates(subset=['도로명주소'], keep='last').reset_index(drop=True)
        
    return concat_df

In [148]:
prac_df = read_coor_data1()
prac_df.shape

  0%|                                                                                           | 0/26 [00:03<?, ?it/s]


(558393, 18)

In [149]:
prac_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,11110,760,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,94,0,NaN,3047,근린생활시설,0,청운효자동,953241.683263,1.954023e+06
1,11110,4933,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,96,0,평안빌,3047,주택,0,청운효자동,953243.258151,1.954034e+06
2,11110,287,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,98,0,청운빌라,3047,주택,0,청운효자동,953240.877646,1.954055e+06
3,11110,1336,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,99,3,풍림팍사이드빌라,3032,주택,1,청운효자동,953195.459364,1.954079e+06
4,11110,6021,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,99,4,경복빌라,3032,주택,1,청운효자동,953193.346779,1.954097e+06


In [150]:
prac_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558393 entries, 0 to 558392
Data columns (total 18 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       558393 non-null  int64  
 1   1       558393 non-null  int64  
 2   2       558393 non-null  int64  
 3   3       558393 non-null  object 
 4   4       558393 non-null  object 
 5   5       558393 non-null  object 
 6   6       558393 non-null  int64  
 7   7       558393 non-null  object 
 8   8       558393 non-null  int64  
 9   9       558393 non-null  int64  
 10  10      558393 non-null  int64  
 11  11      113949 non-null  object 
 12  12      558393 non-null  int64  
 13  13      558390 non-null  object 
 14  14      558393 non-null  int64  
 15  15      558372 non-null  object 
 16  16      557953 non-null  float64
 17  17      557953 non-null  float64
dtypes: float64(2), int64(9), object(7)
memory usage: 80.9+ MB


In [134]:
def read_coor_data2():
    basedir = './좌표데이터2/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.txt'):
            try:
                df = pd.read_csv(basedir + filename, sep='|', header=None, usecols=[1, 2, 3, 4, 5, 7, 8, 23, 24]).drop_duplicates()
            except:
                df = pd.read_csv(basedir + filename, sep='|', header=None, encoding='ansi', usecols=[1, 2, 3, 4, 5, 7, 8, 23, 24])\
                .drop_duplicates()
                
            df['도로명주소'] = df[1] + ' ' + df[2] + ' ' + df[5] + ' ' + df[7].astype('str') + '-' + df[8].astype('str')
            
            df['x좌표'] = df[23]
            df['y좌표'] = df[24]
            
            df = df[['도로명주소', 'x좌표', 'y좌표']]
            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).drop_duplicates(subset=['도로명주소'], keep='last').reset_index(drop=True)
        
    return concat_df

In [135]:
def read_coor_data3():
    basedir = './좌표데이터3/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.csv'):
            df = pd.read_csv(basedir + filename, index_col=0).drop_duplicates()            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).drop_duplicates(subset=['전체주소'], keep='last').reset_index(drop=True)
    concat_df.columns = ['지번주소', 'x좌표', 'y좌표']
    
    return concat_df

In [189]:
def add_0(x):
    if pd.isnull(x) == False:
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x
    else:
        return x
    
def read_coor_data():
    roadname_coor_df1 = read_coor_data1()
    roadname_coor_df2 = read_coor_data2()
    
    roadname_coor_df = pd.concat([roadname_coor_df1, roadname_coor_df2]).drop_duplicates(subset=['도로명주소'], keep='last')\
    .reset_index(drop=True)
    
    landnum_coor_df = read_coor_data3()
        
    return roadname_coor_df, landnum_coor_df

In [139]:
roadname_coor_df, landnum_coor_df = read_coor_data()
print(roadname_coor_df.shape)
print(landnum_coor_df.shape)

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 58.88it/s]


(566869, 3)
(89967, 3)


In [146]:
roadname_coor_df.head(50)

,도로명주소,x좌표,y좌표
0,서울특별시 관악구 난곡로24가길 18-0,948851.369529,1.941318e+06
1,서울특별시 종로구 성균관로15길 33-0,955591.635372,1.954533e+06
2,서울특별시 성북구 인촌로7길 70-0,957563.072605,1.954475e+06
3,서울특별시 서초구 강남대로91길 5-0,957617.535913,1.945960e+06
4,서울특별시 서초구 바우뫼로11길 54-0,958121.727726,1.941641e+06
5,서울특별시 광진구 능동로 297-2,962812.040275,1.950872e+06
6,서울특별시 구로구 개봉로1가길 44-0,942793.009040,1.943123e+06
7,서울특별시 금천구 가산디지털2로 48-0,945417.572753,1.941680e+06
8,서울특별시 강동구 아리수로93가길 408-0,970198.292982,1.952735e+06
9,서울특별시 강동구 아리수로93가길 412-2,970061.222002,1.952674e+06


In [200]:
landnum_coor_df.head()

,지번주소,x좌표,y좌표
0,서울특별시 양천구 신정동 339-0,944806.276643,1.947001e+06
1,서울특별시 강동구 길동 160-0,968954.010468,1.949060e+06
2,서울특별시 양천구 신정동 314-0,944212.602398,1.947199e+06
3,서울특별시 강동구 고덕동 212-0,970202.844451,1.950847e+06
4,서울특별시 양천구 신정동 329-0,944259.661105,1.946646e+06


In [240]:
def replace_invalid_to_nan(df):
    df = df.copy()
    
    both_0_df = df[(df['본번'] == 0) & (df['부번'] == 0)]
        
    df.loc[both_0_df.index, '번지'] = np.nan
    
    return df

In [241]:
main_data_df = replace_invalid_to_nan(main_data_df)

(4668585, 21)

In [268]:
def assign_addr(df):
    df = df.copy()
    
    df['지번주소'] = df['시군구'] + ' ' + df['번지'].apply(add_0)
    df['도로명주소'] = df['시'] + ' ' + df['구'] + ' ' + df['도로명'].apply(add_0)
    
    both_nan_df = df[(df['번지'].isna()) & (df['도로명'].isna())]
    not_0_df = both_nan_df[both_nan_df['본번'] != 0]
    
    df.loc[not_0_df.index, '지번주소'] = not_0_df['시군구'] + ' ' + not_0_df['본번'].astype('str') + '-' + not_0_df['부번'].astype('str')
    
    return df

In [269]:
main_data_df = assign_addr(main_data_df)

MemoryError: Unable to allocate 285. MiB for an array with shape (8, 4668585) and data type object

In [266]:
main_data_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,지번주소,도로명주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,55000,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,58250,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,60000,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0


## Selecting addresses in main data that are not in coordinates data

In [250]:
unique_landnum_addr = main_data_df['지번주소'].unique()
unique_roadname_addr = main_data_df['도로명주소'].unique()

In [261]:
prac_df = main_data_df[(main_data_df['지번주소'].isna()) & (main_data_df['도로명주소'].isna())]
prac_df.shape

(908, 21)

In [262]:
prac_df.head(20)

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,지번주소,도로명주소
1566422,서울특별시 마포구 공덕동,NaN,175,100,공덕동주택,18.90,200604,5,11000,-1,1990,NaN,582.010582,2006,4,서울특별시,마포구,매매,연립다세대,NaN,NaN
1566423,서울특별시 마포구 공덕동,NaN,175,100,공덕동주택,15.66,200609,20,9300,-1,1990,NaN,593.869732,2006,9,서울특별시,마포구,매매,연립다세대,NaN,NaN
1566424,서울특별시 마포구 공덕동,NaN,175,100,공덕동주택,18.90,200612,4,12500,-1,1990,NaN,661.375661,2006,12,서울특별시,마포구,매매,연립다세대,NaN,NaN
1682025,서울특별시 마포구 공덕동,NaN,175,231,공덕동주택,60.11,200803,5,20500,3,2004,NaN,341.041424,2008,3,서울특별시,마포구,매매,연립다세대,NaN,NaN
1682026,서울특별시 마포구 공덕동,NaN,175,100,공덕동주택,16.50,200803,24,16956,2,1990,NaN,1027.636364,2008,3,서울특별시,마포구,매매,연립다세대,NaN,NaN
1682027,서울특별시 마포구 공덕동,NaN,175,100,공덕동주택,14.50,200804,5,13500,2,1990,NaN,931.034483,2008,4,서울특별시,마포구,매매,연립다세대,NaN,NaN
1838377,서울특별시 구로구 천왕동,NaN,0,0,네이처힐,24.14,201406,2,12444,8,2014,NaN,515.492958,2014,6,서울특별시,구로구,매매,오피스텔,NaN,NaN
1838378,서울특별시 구로구 천왕동,NaN,0,0,네이처힐,24.14,201408,15,13191,4,2014,NaN,546.437448,2014,8,서울특별시,구로구,매매,오피스텔,NaN,NaN
1838379,서울특별시 구로구 천왕동,NaN,0,0,네이처힐,24.14,201408,15,13191,7,2014,NaN,546.437448,2014,8,서울특별시,구로구,매매,오피스텔,NaN,NaN
1838380,서울특별시 구로구 천왕동,NaN,0,0,네이처힐,24.14,201408,19,13191,6,2014,NaN,546.437448,2014,8,서울특별시,구로구,매매,오피스텔,NaN,NaN
